In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras import optimizers

In [3]:
train_dir = 'chest_xray-20221112T154233Z-001/chest_xray/train'
test_dir = 'chest_xray-20221112T154233Z-001/chest_xray//test'

In [4]:
size = [224,224,3]

In [5]:
img_aug = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, fill_mode='nearest')

In [6]:
train_data = img_aug.flow_from_directory(train_dir, target_size=(224,224), batch_size=16, class_mode='categorical')
test_data = img_aug.flow_from_directory(test_dir, target_size=(224,224), batch_size=16, class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [10]:
def build_model1():
    vgg16 = tf.keras.applications.VGG16(weights="imagenet", include_top = False, input_shape = size)
    x = Flatten()(vgg16.output)
    x = Dense(64, activation = 'relu')(x)
    x = Dense(32, activation = 'relu')(x)
    x = Dense(16, activation = 'relu')(x)
    mod = Dense(4, activation = 'relu')(x)
    model = Model(inputs = vgg16.input, outputs = mod)
    return model


def build_model2():
    mbnet = tf.keras.applications.MobileNetV2(weights="imagenet", include_top = False, input_shape = size)
    x = Flatten()(mbnet.output)
    x = Dense(64, activation = 'relu')(x)
    x = Dense(32, activation = 'relu')(x)
    x = Dense(16, activation = 'relu')(x)
    mod = Dense(4, activation = 'relu')(x)
    model = Model(inputs = mbnet.input, outputs = mod)
    return model

inputs = tf.keras.layers.Input(shape=size)
model1 = build_model1()
model2 = build_model2()
combined = tf.keras.layers.Concatenate()([model1(inputs), model2(inputs)])
outputs = Dense(2, activation = 'softmax')(combined)
model = Model(inputs, outputs)

In [11]:
model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(train_data, epochs = 5, verbose = 1)

Epoch 1/5
326/326 [==============================] - 4251s 13s/step - loss: 0.1838 - accuracy: 0.9285
Epoch 2/5
326/326 [==============================] - 4009s 12s/step - loss: 0.0894 - accuracy: 0.9636
Epoch 3/5
326/326 [==============================] - 4089s 13s/step - loss: 0.0779 - accuracy: 0.9720
Epoch 4/5
326/326 [==============================] - 3896s 12s/step - loss: 0.0661 - accuracy: 0.9749
Epoch 5/5
326/326 [==============================] - 4034s 12s/step - loss: 0.0593 - accuracy: 0.9772


In [13]:
preds = model.predict(test_data)
true_classes = test_data.classes
mod_pred_classes = np.argmax(preds, axis=1)

39/39 [==============================] - 124s 3s/step


In [14]:
from sklearn.metrics import accuracy_score

mod_acc = accuracy_score(true_classes, mod_pred_classes)
print("Model Accuracy: {:.4}%".format(mod_acc * 100))

Model Accuracy: 60.74%


In [21]:
from sklearn.metrics import classification_report
print(classification_report(mod_pred_classes,true_classes))

              precision    recall  f1-score   support

           0       0.06      0.36      0.10        39
           1       0.94      0.62      0.75       585

    accuracy                           0.61       624
   macro avg       0.50      0.49      0.43       624
weighted avg       0.88      0.61      0.71       624

